In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql import Row
from numpy import prod
import datetime
import pandas

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.executor.memory", "1g") \
    .getOrCreate()

sc = spark.sparkContext

from pyspark.streaming import StreamingContext

streamContext = StreamingContext(sc,1)

def consumption(mpg):
    if float(mpg)>22:
        return('high_consumption')
    else:
        return('low_consumption')

consumption_udf = udf(consumption, StringType())

lines = streamContext.socketTextStream("localhost", 9000)

def my_processing(rdd):
    my_lines = rdd.map(lambda l: l.split(","))\
    .map(lambda p: Row(time=p[0],name=p[1], mpg=p[2], cyl=p[3], \
                       disp=p[4], hp=p[5], drat=p[6], wt=p[7], qsec=p[8], vs=p[9],\
                      am=p[10], gear=p[11], carb=p[12]))
    mtcars = my_lines.toDF(["am","carb","cyl","disp","drat","gear","hp","mpg","name","qsec","time","vs","wt"])
    mtcars = mtcars.withColumn('mpg_per_hp', mtcars.mpg * mtcars.hp )
    mtcars = mtcars.withColumn('mpg_', consumption_udf(mtcars.mpg))
    adat= mtcars.toPandas() 
    print(adat)
    print('############################################################################')
    
lines.foreachRDD(my_processing)

streamContext.start()

  am carb cyl   disp  drat gear   hp   mpg        name  qsec      time vs  \
0  0    3   8  275.8  3.07    3  180  16.4  Merc 450SE  17.4  16:11:09  0   

     wt  mpg_per_hp             mpg_  
0  4.07      2952.0  low_consumption  
############################################################################
   am carb cyl   disp  drat gear   hp   mpg                name   qsec  \
0   0    2   8    360  3.15    3  175  18.7   Hornet Sportabout  17.02   
1   1    1   4   78.7  4.08    4   66  32.4            Fiat 128  19.47   
2   1    6   6    145  3.62    5  175  19.7        Ferrari Dino   15.5   
3   0    3   8  275.8  3.07    3  180  17.3          Merc 450SL   17.6   
4   1    4   6    160   3.9    4  110    21           Mazda RX4  16.46   
5   0    4   8    350  3.73    3  245  13.3          Camaro Z28  15.41   
6   0    4   8    472  2.93    3  205  10.4  Cadillac Fleetwood  17.98   
7   1    1   4     79  4.08    4   66  27.3           Fiat X1-9   18.9   
8   0    4   8    472  2